In [ ]:
! ls

In [ ]:
! git clone https://huggingface.co/openai-community/gpt2

Cloning into 'gpt2'...
remote: Enumerating objects: 87, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 87 (delta 0), reused 0 (delta 0), pack-reused 84
Unpacking objects: 100% (87/87), 1.65 MiB | 6.61 MiB/s, done.
Filtering content: 100% (11/11), 5.23 GiB | 48.40 MiB/s, done.


# **Загрузка модели**

In [1]:
import transformers
import torch

In [2]:
device = 'cuda'

In [3]:
model = transformers.AutoModelForCausalLM.from_pretrained(
    'gpt2',
    torch_dtype=torch.float64,
)

model.to(device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [4]:
tokenizer = transformers.AutoTokenizer.from_pretrained("gpt2")
tokenizer.add_special_tokens({'pad_token': "<|endoftext|>"})

1

# **Задание 1. LayerNorm vs RMSNorm**

для примера ниже нужно релизовать подсчет var и вычислить входной вектор в случае LayerNorm и RMSNorm



1) LayerNorm:

the paper: https://arxiv.org/abs/1607.06450




![](https://drive.google.com/uc?export=view&id=1rUfBxhGNGBw98EGWlCq7rOKYUEoyld2i)

2) RMSNorm:

the paper: https://arxiv.org/pdf/1910.07467v1.pdf


![](https://drive.google.com/uc?export=view&id=1s5Ed7ZRsZ1KvIT6Np-zSiBhckYc1NfWQ)

In [ ]:
example = torch.Tensor([[1, 2, 3, 4, 5], [2, 3, 4, 5, 6]])

In [ ]:
example

tensor([[1., 2., 3., 4., 5.],
        [2., 3., 4., 5., 6.]])

In [ ]:
class LayerNorm:
  def __init__(self, eps: float = 1e-5):
    self.eps = eps

  def __call__(self, input: torch.Tensor):
    mean = (input.sum(axis=1) / input.shape[1]).reshape(-1, 1)
    var = # your code
    return (input - mean) / (torch.sqrt(var + self.eps))

In [ ]:
layer_norm = LayerNorm()
result = layer_norm(example)
result

In [ ]:
### answer: result[0][0]

In [ ]:
class RMSNorm:
  def __init__(self, eps: float = 1e-5):
    self.eps = eps

  def __call__(self, input: torch.Tensor):
    var = # your code
    return input / torch.sqrt(var + self.eps)

In [21]:
rms_norm = RMSNorm()
result = rms_norm(example)
result

In [21]:
### answer: result[0][0]

# Задание 2. Обучение модели.

1) необходимо подобрать такой lr, при котором лосс будет иметь тенденцию к снижению (скользящее срденее которого убывает)



In [5]:
import pandas as pd
from torch import nn
import transformers
import torch

In [6]:
sft_df = pd.read_parquet('argilla_df.parquet')

In [9]:
from torch.utils.data import Dataset, DataLoader, Sampler
from typing import Dict, Sequence, List, Iterator, Tuple

IGNORE_INDEX: int = -100
pad_token_id = 60000

def _tokenize_fn(text: str, max_length: int, tokenizer: transformers.PreTrainedTokenizer) -> List[int]:
    return tokenizer(
        text,
        max_length=max_length,
        truncation=True,
        add_special_tokens=False,
    )['input_ids']

class SupervisedDataset(Dataset):
    """Dataset for supervised fine-tuning."""
    def __init__(self, dataset, tokenizer: transformers.PreTrainedTokenizer):
        super(SupervisedDataset, self).__init__()
        self.tokenizer = tokenizer

        bos_token = tokenizer.bos_token if tokenizer.bos_token is not None else ''
        self.sources = [f"{bos_token}{example[0]}" for example in dataset]
        self.targets = [f"{example[1]}{tokenizer.eos_token}" for example in dataset]

    def __len__(self) -> int:
        return len(self.sources)

    def padding(self, ids: torch.tensor, max_length: int) -> torch.tensor:
        return

    def __getitem__(self, i: int) -> Dict[str, List[int]]:
        source_ids = _tokenize_fn(self.sources[i], self.tokenizer.model_max_length, self.tokenizer)
        target_ids = _tokenize_fn(self.targets[i], self.tokenizer.model_max_length - len(source_ids), self.tokenizer)
        input_ids = torch.tensor(source_ids + target_ids)

        input_pad_len = self.tokenizer.model_max_length - len(input_ids)
        input_ids_pad = torch.tensor(source_ids + target_ids + [self.tokenizer.pad_token_id]*input_pad_len)
        input_att_mask = torch.tensor([[1]*len(input_ids) + [0]*input_pad_len])

        labels = torch.tensor([IGNORE_INDEX] * len(source_ids) + target_ids + [pad_token_id]*input_pad_len)

        return dict(input_ids=input_ids_pad, input_att_mask=input_att_mask,
                    labels=labels)


In [ ]:
tokenizer.pad_token_id

50256

In [8]:
train_batch = SupervisedDataset(
    dataset=sft_df[:200][['prompt', 'original_response']].values,
    tokenizer=tokenizer,
)

In [9]:
train_batch_loader = DataLoader(
    dataset=train_batch,
    batch_size=1,
    shuffle=False,
)

In [21]:
loss = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=model.parameters(), lr= ### Your value)
N_ITERATIONS = 100
cur_iteration = 0
loss_values = []
for x in train_batch_loader:
    if cur_iteration == N_ITERATIONS:
      break

    model.train()
    #################
    input_tokens = x['input_ids'].to(device)
    attention_mask = x['input_att_mask'].to(device)
    labels = x['labels'].clone().to(device)
    out_logits = model(input_ids=input_tokens, attention_mask=attention_mask).logits

    ### не затираем токены начала и конца, чтобы модель училась останавливаться ###
    labels[labels == pad_token_id] = -100

    loss_value = loss(out_logits.permute(0, 2, 1), labels)
    #################
    print(f"Loss value: {loss_value.item()}")
    loss_value.backward()
    optimizer.step()

    model.eval()
    cur_iteration += 1
    loss_values.append(loss_value.cpu().detach().numpy())


### 2) обратите внимание на то, как выглядит лосс

In [21]:
import matplotlib.pyplot as plt
import numpy as np

# Data for plotting
t = [i for i in range(len(loss_values))]
s = loss_values

fig, ax = plt.subplots()
ax.plot(t, s)

ax.set(xlabel='step', ylabel='loss value',
       title='Train Loss')
ax.grid()

plt.show()

## 3) попробуйте явно прописать задание модели, то есть в качестве промпта задать: Тебе необходимо ответить на вопрос, приведенные ниже: ....

вставьте следующий текст перед самим вопросом: "You must answer the question correctly. Question: "

вставьте следующие текст между самим вопросом и правильным ответом на вопрос: "\n###answer: "

In [21]:
from torch.utils.data import Dataset, DataLoader, Sampler
from typing import Dict, Sequence, List, Iterator, Tuple

IGNORE_INDEX: int = -100
pad_token_id = 60000

def _tokenize_fn(text: str, max_length: int, tokenizer: transformers.PreTrainedTokenizer) -> List[int]:
    return tokenizer(
        text,
        max_length=max_length,
        truncation=True,
        add_special_tokens=False,
    )['input_ids']

class SupervisedDataset(Dataset):
    """Dataset for supervised fine-tuning."""
    def __init__(self, dataset, tokenizer: transformers.PreTrainedTokenizer):
        super(SupervisedDataset, self).__init__()
        self.tokenizer = tokenizer

        bos_token = tokenizer.bos_token if tokenizer.bos_token is not None else ''
        self.sources = [f"{bos_token}{example[0]}" for example in dataset]
        self.targets = [f"{example[1]}{tokenizer.eos_token}" for example in dataset]

    def __len__(self) -> int:
        return len(self.sources)

    def padding(self, ids: torch.tensor, max_length: int) -> torch.tensor:
        return

    def __getitem__(self, i: int) -> Dict[str, List[int]]:
        source_ids = _tokenize_fn(self.sources[i], self.tokenizer.model_max_length, self.tokenizer)
        target_ids = _tokenize_fn(self.targets[i], self.tokenizer.model_max_length - len(source_ids), self.tokenizer)
        input_ids = torch.tensor(source_ids + target_ids)

        input_pad_len = self.tokenizer.model_max_length - len(input_ids)
        input_ids_pad = torch.tensor(source_ids + target_ids + [self.tokenizer.pad_token_id]*input_pad_len)
        input_att_mask = torch.tensor([[1]*len(input_ids) + [0]*input_pad_len])

        labels = torch.tensor([IGNORE_INDEX] * len(source_ids) + target_ids + [pad_token_id]*input_pad_len)

        return dict(input_ids=input_ids_pad, input_att_mask=input_att_mask,
                    labels=labels)


In [ ]:
### вставьте следующий текст перед самим вопросом: "You must answer the question correctly. Question: "
### вставьте следующие текст между самим вопросом и правильным ответом на вопрос: "\n###answer: "

In [22]:
train_batch = SupervisedDataset(
    dataset=sft_df[:200][['prompt', 'original_response']].values,
    tokenizer=tokenizer,
)

In [23]:
train_batch_loader = DataLoader(
    dataset=train_batch,
    batch_size=1,
    shuffle=False,
)

In [25]:
for x in train_batch_loader:
  break

In [26]:
### answer: x['input_ids'][0][:10]

In [10]:
model = transformers.AutoModelForCausalLM.from_pretrained(
    'gpt2',
    torch_dtype=torch.float64,
)

model.to(device)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [26]:
loss = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=model.parameters(), lr= #Your value)
N_ITERATIONS = 100
cur_iteration = 0
loss_values = []
for x in train_batch_loader:
    if cur_iteration == N_ITERATIONS:
      break

    model.train()
    #################
    input_tokens = x['input_ids'].to(device)
    attention_mask = x['input_att_mask'].to(device)
    labels = x['labels'].clone().to(device)
    out_logits = model(input_ids=input_tokens, attention_mask=attention_mask).logits

    ### не затираем токены начала и конца, чтобы модель училась останавливаться ###
    labels[labels == pad_token_id] = -100

    loss_value = loss(out_logits.permute(0, 2, 1), labels)
    #################
    print(f"Loss value: {loss_value.item()}")
    loss_value.backward()
    optimizer.step()

    model.eval()
    cur_iteration += 1
    loss_values.append(loss_value.cpu().detach().numpy())

In [26]:
import matplotlib.pyplot as plt
import numpy as np

# Data for plotting
t = [i for i in range(len(loss_values))]
s = loss_values

fig, ax = plt.subplots()
ax.plot(t, s)

ax.set(xlabel='step', ylabel='loss value',
       title='Train Loss')
ax.grid()

plt.show()